In [64]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.nn import Module
from pprint import pprint

from sentence_transformers import SentenceTransformer
st_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


class inspect_func(Module):
    def __call__(self, *args, **kwargs):
        pprint(args)
        pprint(kwargs)
        print(kwargs["encoder_hidden_states"].shape)


In [65]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")




In [70]:
class CustomT5(T5ForConditionalGeneration):
    def __init__(self, *args, st_model=None, tokenizer=None, **kwargs):    
        super().__init__(*args, **kwargs)
        self.encoder = self.set_encoder(self.encoder, st_model, tokenizer)
        #self.decoder = inspect_func()
        self.t5_tokenizer = tokenizer


   # def __call__(self, text):
    #    tokenized = self.t5_tokenizer(text, return_tensors="pt")
    #    return super().__call__(**tokenized)

    #def forward(self, text, labels=None):
     #   tokenized = self.t5_tokenizer(text, return_tensors="pt")
     #   return super().forward(**tokenized, labels=labels)

    def generate(self, text):
        tokenized = self.t5_tokenizer(text, return_tensors="pt")
        return super().generate(**tokenized)


    def set_encoder(self, standard_encoder, st_model, tokenizer):
        class CustomEncoder(Module):
            def __init__(self, *args, **kwargs):    
                super().__init__(*args, **kwargs)
                self.t5_encoder = standard_encoder
                self.st_model = st_model
                self.tokenizer = tokenizer

            def forward(self, text):

                print(text)
                print("BLAH")
                tokenized = self.tokenizer(text)
                print("hello\n"*30)
                encoder_output = self.t5_encoder(**tokenized)
                print("we are here")
                return(encoder_output)
        
        return CustomEncoder()
                





In [71]:
from IPython.display import clear_output
model = CustomT5.from_pretrained("t5-small", st_model=st_model, tokenizer=tokenizer)
clear_output()

Some weights of the model checkpoint at t5-small were not used when initializing CustomT5: ['encoder.block.0.layer.0.SelfAttention.k.weight', 'encoder.block.4.layer.1.layer_norm.weight', 'encoder.block.0.layer.0.SelfAttention.v.weight', 'encoder.block.3.layer.1.layer_norm.weight', 'encoder.block.1.layer.1.DenseReluDense.wo.weight', 'encoder.block.1.layer.0.SelfAttention.v.weight', 'encoder.block.5.layer.0.SelfAttention.v.weight', 'encoder.final_layer_norm.weight', 'encoder.block.4.layer.0.SelfAttention.q.weight', 'encoder.block.5.layer.1.DenseReluDense.wi.weight', 'encoder.block.4.layer.0.SelfAttention.o.weight', 'encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.1.layer.1.layer_norm.weight', 'encoder.block.5.layer.1.layer_norm.weight', 'encoder.block.3.layer.1.DenseReluDense.wo.weight', 'encoder.block.3.layer.0.SelfAttention.q.weight', 'encoder.block.4.layer.0.layer_norm.weight', 'encoder.block.4.layer.0.SelfAttention.v.weight', 'encoder.block.3.lay

In [72]:
generated = model.generate("Who is the prime minister of Chile?")
tokenizer.decode(generated[0], skip_special_tokens=True)

TypeError: forward() got an unexpected keyword argument 'return_dict'